# #5. Black-Litterman model

In [1]:
import matplotlib.pylab as plt
import numpy as np
from numpy.linalg import inv
import pandas as pd
from pandas_datareader import data as web
from scipy.optimize import minimize

In [2]:
# 최적의 비중 계산을 위한 목적함수
# obj. funciton for optimizing weights
def solveWeights(R, C, rf):
    def obj(W, R, C, rf):
        mean = sum (R * W)
        var = np.dot(np.dot(W, C), W)
        
        # Sharpe's ratio as utility function: want to maximize
        util = (mean -rf) / np.sqrt(var)
        return 1 / util
    
    # number of assets
    n = len(R)
    
    # initial weights
    W = np.ones([n]) / n
    
    # doesn't allow short-selling nor borrowing
    # 비중 범위는 0-100% 사이: 공매도나 차입조건이 없음
    bnds = [(0., 1.) for i in range(n)]
    
    # constraints: sum of weights = 1
    cons = ({'type': 'eq', 'fun': lambda W: sum(W) - 1.})

    res = minimize(obj, W, (R, C, rf), method='SLSQP', constraints=cons, bounds=bnds)

    if not res.success:
        raise BaseException(res.message)
    
    return res.x

In [3]:
# 무위험 수익률, 수익률, Cov로 효율적 투자선 계산
# calculating Efficient Frontier via risk-less rate, return rate and the Covariance
def solveFrontier(R, C, rf):

    # 최적 비중 계산을 위한 목적함수
    def obj(W, R, C, r):

        # 주어진 수익률에서 분산을 최소화하는 비중 계산
        mean = sum(R * W)
        var = np.dot(np.dot(W, C), W)

        # 최적화 제약조건 페널티
        penalty = 100 * abs(mean - r)
        return var + penalty
    
    frontier_mean, frontier_var = [ ], [ ]
    n = len(R)
    
    # 수익률 최저에서 최대 사이를 반복
    for r in np.linspace(min(R), max(R), num=20):
        W = np.ones([n]) / n

        bnds = [(0, 1) for i in range(n)]
        cons = [{'type': 'eq', 'fun': lambda W: sum(W) - 1.}]

        res = minimize(obj, W, (R, C, r), method='SLSQP', constraints=cons, bounds=bnds)

        if not res.success:
            raise BaseException(res.message)
        
        frontier_mean.append(r)
        frontier_var.apend(np.dot(np.dot(res.x, C), res.x))

    return np.array(frontier_mean), np.array(frontier_var)

In [4]:
# 효율적 포트폴리오 최적화
def optimize_frontier(R, C, rf):

    # tangency portfolio 계산
    W = solveWeights(R, C, rf)

    tan_mean = sum(R * W)
    tan_var = np.dot(np.dot(W, C), W)

    # 효율적 포트폴리오 계산
    eff_mean, eff_var = solveFrontier(R, C, rf)
    
    # dict 타입으로 리턴
    return {'weights':W, 'tan_mean':tan_mean, 'tan_var':tan_var, 'eff_mean':eff_mean, 'eff_var':eff_var}

In [ ]:
# 자산에 대한 투자자의 전망 (행렬)과 전망의 기대수익률 행렬
def CreateMatrixPQ(names, views):
    r, c = len(views), len(names)

    # views[i][3] = exp. return(기대 수익률)
    Q = [views[i][3] for i in range(r)]

In [ ]:
# read the data
tickers = ['PFE','INTC','NFLX','JPM','XOM','GOOG','JNJ','AAPL','AMZN']
cap = {'PFE':201102000000,'INTC':257259000000,'NFLX':184922000000,
       'JPM':272178000000,'XOM':178228000000,'GOOG':866683000000,
       'JNJ':403335000000,'AAPL':1208000000000,'AMZN':1178000000000
      }

prices, caps = [ ], [ ]

for s in tickers:
    pxclose = web.DataReader(s, data_source='yahoo', start='01-01-2018', end='31-12-2019')['Adj Close']
    prices.append(list(pxclose))
    caps.append(cap[s])

In [ ]:
n = len(tickers)
W = np.array(caps) / sum(caps)
prices = np.matrix(prices)

rows, cols = prices.shape
returns = np.empty([rows, cols - 1])
for r in range(rows):
    for c in range(cols - 1):
        p0, p1 = prices[r, c], prices[r, c + 1]
        returns[r, c] = (p1 / p0) - 1

expreturns = np.array([])
for r in range(rows):
    expreturns = np.append(expreturns, np.mean(returns[r]))

covars = np.cov(returns)
R = (1 + expreturns) ** 250 - 1
C = covars * 250

rf = .015

In [ ]:
expreturns